# Trustcall

- Memory management tool calling, built using langgraph
- Updates schemas without losing contents.
- LLM powered JSON patching - taking leverage from LLM's structured output mastery.


In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model='gemini-2.5-pro',temperature=0)

In [86]:
from pydantic import BaseModel, Field
from typing import Literal, Optional, Annotated

from trustcall import create_extractor

Interests = Literal['programming', 'ai', 'art', 'lifestyle', 'finance']

class Publication(BaseModel):
    """Publication - Defining the Article"""
    id: str = Field(description="Id of the publication")
    title: str = Field(description='Title of the Publication')
    summary: str = Field(description='Summary of the article in about 100 words')
    author: str = Field(description='Author Full name')
    url: str = Field(description="Link for the publication")

class User(BaseModel):
    interests: list[Interests] | None = Field(description="List/Array of interests: Example ['science']", default=None)
    reads: list[Publication] | None = Field(description='list/array of publications user reads', default=None)
    recommended_reads: list[Publication] | None = Field(description="Recommended list of Publications based on interests", default=None)


In [87]:
from uuid import uuid4

USERS: dict[str, User] = {
    'alice': User(interests=['programming', 'ai']),
    'bob': User(interests=['finance'])
}

In [88]:
USERS

{'alice': User(interests=['programming', 'ai'], reads=None, recommended_reads=None),
 'bob': User(interests=['finance'], reads=None, recommended_reads=None)}

In [89]:
from langgraph.store.memory import InMemoryStore
in_memory_store = InMemoryStore()

existing_user = USERS['alice']
users_ns: tuple[Literal['users'], str] = ('users', 'alice')
user_id = str(uuid4())
in_memory_store.put(
    users_ns,
    user_id,
    existing_user.model_dump()
)


In [124]:
from requests import get, Response
from langchain_core.tools import tool

@tool
def get_posts(category: str = None, size: int = 3) -> list[Publication]:
    """Get posts from API based on category"""
    if category is None:
        return Publication(
            title='Welcome post; Know more about lifestyle, ai, programming, art, finance',
            id='lifestyle-999',
            summary='Sample summary',
            author='Alice',
            url='dummy.com'
        )

    publications: list[Publication] = get(f"""http://localhost:4001/feed?category={category}""", headers = {'Cache-Control': 'no-cache'}).json()
    return publications[:size]



In [125]:
res: list[Publication] = get_posts(category='ai')
res

/var/folders/13/6bcjxw75117_6m_84gvdp5080000gn/T/ipykernel_97951/3770273754.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res: list[Publication] = get_posts(category='ai')


TypeError: BaseTool.__call__() got an unexpected keyword argument 'category'

In [100]:
recommendations_ns = ('recommendations', 'ai')
in_memory_store.put(
    recommendations_ns,
    user_id,
    {
        'interests': existing_user.interests,
        'recommended_reads': res 
    }
)

In [101]:
memories = in_memory_store.search(users_ns)
memories # list

[Item(namespace=['users', 'alice'], key='8e1eaa1a-788f-4b16-8ea5-a0db99d4f368', value={'interests': ['programming', 'ai'], 'reads': None, 'recommended_reads': None}, created_at='2025-08-04T02:50:52.501450+00:00', updated_at='2025-08-04T02:50:52.501453+00:00', score=None)]

In [102]:
pubs = in_memory_store.search(recommendations_ns)
pubs

[Item(namespace=['recommendations', 'ai'], key='8e1eaa1a-788f-4b16-8ea5-a0db99d4f368', value={'interests': ['programming', 'ai'], 'recommended_reads': [{'id': 'ai-301', 'title': 'Fine-Tuning LLMs: What You Need to Know', 'author': 'Dr. Emily Zhou', 'summary': 'Key considerations and best practices for LLM fine-tuning in 2025.', 'url': 'https://medium.com/ai/fine-tuning-llms-efghi'}, {'id': 'ai-302', 'title': 'The Rise of Responsible AI', 'author': 'Martin Lewis', 'summary': 'Balancing innovation and ethics in artificial intelligence.', 'url': 'https://medium.com/ai/rise-of-responsible-ai-fghij'}, {'id': 'ai-303', 'title': 'AI Agents: Hype, Hope, and Reality', 'author': 'Cindy Alvarez', 'summary': 'Parsing fact from fiction in the world of autonomous AI agents.', 'url': 'https://medium.com/ai/ai-agents-hype-vs-reality-plokm'}]}, created_at='2025-08-04T02:54:37.733332+00:00', updated_at='2025-08-04T02:54:37.733334+00:00', score=None)]

In [103]:
memories[0].dict()

{'namespace': ['users', 'alice'],
 'key': '8e1eaa1a-788f-4b16-8ea5-a0db99d4f368',
 'value': {'interests': ['programming', 'ai'],
  'reads': None,
  'recommended_reads': None},
 'created_at': '2025-08-04T02:50:52.501450+00:00',
 'updated_at': '2025-08-04T02:50:52.501453+00:00',
 'score': None}

In [119]:
USER_PROFILE_INSTRUCTION = """You are a helpful assistant with memory that provides information about the user. 
If you have memory for this user, use it to personalize your responses. Note: From the conversation, try identifying the category with the known interests
Here is the memory (it may be empty): {memory}"""

CREATE_OR_UPDATE_USER_PROFILE_INSTRUCTION = """You are collecting information about the user to personalize your responses.

CURRENT USER INFORMATION:
<profile>
{profile}
</profile>

INSTRUCTIONS:
1. Review the chat history below carefully
2. Identify new information about the user, such as:
    - Personal details (name, location)
    - Preferences (likes, dislikes)
    - Interests and Reads
    - Past experiences
    - Goals or future plans
    - Keep adding attributes to the profile as you discover things
3. Merge any new information with existing memory
4. Format the memory as a clear, bulleted list
5. If new information conflicts with existing memory, keep the most recent version

Remember: 
1. The goal is to create/update user profile information and not to answer the questions based on chat history.
2. Only include factual information directly stated by the user. Do not make assumptions or inferences.

Example Output: "**User Information:**\n- User's name is Bob.\n- Likes to read about Programming."

please update the user information based on the chat history:
"""

CREATE_OR_UPDATE_RECOMMENDATIONS_INSTRUCTION = """You are a helpful recommendation assistant for recommending publications based on user interests.

Following is the list of current user interests (maybe empty as well).

CURRENT USER INTERESTS: {categories}

CURRENT RECOMMENDED READS: {rec_reads}

Steps to do:
1. If its a new user, meaning interests are empty get 1 recommendation for all five categories, so we get to know the users interests
2. Get three publications using the `get_posts` tool with `category` as the one user is interested in.
3. Send an updated list of `recommended_reads` for the user based on the interests and previous reads if any.
"""

In [118]:
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from typing import Literal


conversation = [
    HumanMessage(content="Hi, I just finished reading Gen AI applications")
]

In [120]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4')
user_profile_extractor = create_extractor(
    llm,
    tools=[User, Publication],
    tool_choice="User",
    enable_inserts=True
)



user_profile = user_profile_extractor.invoke({
    "messages": [SystemMessage(content=USER_PROFILE_INSTRUCTION.format(memory=memories[0].dict()['value']))] + conversation,
    # "existing": memories[0].dict()['value']  # Optionally provide current todos for update
})

user_profile['responses']

[User(interests=None, reads=[Publication(id='001', title='Gen AI applications', summary='', author='', url='')], recommended_reads=None)]

In [107]:
publications_extractor = create_extractor(
    llm,
    tools=[User, Publication],
    tool_choice="Publication",
    enable_inserts=True
)

publications= publications_extractor.invoke({
    "messages": [SystemMessage(content=CREATE_OR_UPDATE_USER_PROFILE_INSTRUCTION.format(profile=memories[0].dict()['value']))] + conversation,
    # "existing": memories[0].dict()['value']  # Optionally provide current todos for update
})

publications['responses']

[Publication(id='1', title='Clean Code', summary='A handbook of Agile software craftsmanship', author='Robert C. Martin', url='')]

In [147]:
rec_mem = in_memory_store.search(recommendations_ns)
rec_mem[0].dict()['value']

{'interests': ['programming', 'ai'],
 'recommended_reads': [{'id': 'ai-301',
   'title': 'Fine-Tuning LLMs: What You Need to Know',
   'author': 'Dr. Emily Zhou',
   'summary': 'Key considerations and best practices for LLM fine-tuning in 2025.',
   'url': 'https://medium.com/ai/fine-tuning-llms-efghi'},
  {'id': 'ai-302',
   'title': 'The Rise of Responsible AI',
   'author': 'Martin Lewis',
   'summary': 'Balancing innovation and ethics in artificial intelligence.',
   'url': 'https://medium.com/ai/rise-of-responsible-ai-fghij'},
  {'id': 'ai-303',
   'title': 'AI Agents: Hype, Hope, and Reality',
   'author': 'Cindy Alvarez',
   'summary': 'Parsing fact from fiction in the world of autonomous AI agents.',
   'url': 'https://medium.com/ai/ai-agents-hype-vs-reality-plokm'}]}

In [144]:
class Spy:
    def __init__(self):
        self.called_tools = []

    def __call__(self, run):
        # Collect information about the tool calls made by the extractor.
        q = [run]
        while q:
            r = q.pop()
            if r.child_runs:
                q.extend(r.child_runs)
            if r.run_type == "chat_model":
                self.called_tools.append(
                    r.outputs["generations"][0][0]["message"]["kwargs"]["tool_calls"]
                )

# Initialize the spy
spy = Spy()

In [145]:
def extract_tool_info(tool_calls, schema_name):
    """Extract information from tool calls for both patches and new memories.
    
    Args:
        tool_calls: List of tool calls from the model
        schema_name: Name of the schema tool (e.g., "User", "Publication")
    """

    # Initialize list of changes
    changes = []
    
    for call_group in tool_calls:
        for call in call_group:
            if call['name'] == 'PatchDoc':
                changes.append({
                    'type': 'update',
                    'doc_id': call['args']['json_doc_id'],
                    'planned_edits': call['args']['planned_edits'],
                    'value': call['args']['patches'][0]['value']
                })
            elif call['name'] == schema_name:
                changes.append({
                    'type': 'new',
                    'value': call['args']
                })

    # Format results as a single string
    result_parts = []
    for change in changes:
        if change['type'] == 'update':
            result_parts.append(
                f"Document {change['doc_id']} updated:\n"
                f"Plan: {change['planned_edits']}\n"
                f"Added content: {change['value']}"
            )
        else:
            result_parts.append(
                
                f"New {schema_name} created:\n"
                f"Content: {change['value']}"
            )
    
    return "\n\n".join(result_parts)

In [157]:
llm_with_tools = llm.bind_tools([get_posts])


recommendations_extractor = create_extractor(
    llm_with_tools,
    tools=[User, get_posts],
    # tool_choice='get_posts',
    enable_inserts=True
)

publications= publications_extractor.invoke({
    "messages": [SystemMessage(
        content=CREATE_OR_UPDATE_RECOMMENDATIONS_INSTRUCTION.format(
            categories=memories[0].dict()['value'],
            rec_reads=rec_mem[0].dict()['value']['recommended_reads']
            )
        )] + conversation,
    # "existing": rec_mem[0].dict()['value']['recommended_reads']  # Optionally provide current todos for update
})

tool_calls = publications['messages'][-1].tool_calls

    # Extract the changes made by Trustcall and add the the ToolMessage returned to task_mAIstro
todo_update_msg = extract_tool_info(spy.called_tools, 'get_tools')
print({"messages": [{"role": "tool", "content": todo_update_msg, "tool_call_id":tool_calls[0]['id']}]})

{'messages': [{'role': 'tool', 'content': '', 'tool_call_id': 'call_AAa8blCpCWKH8zFXGZdBu0aL'}]}


In [158]:
publications

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AAa8blCpCWKH8zFXGZdBu0aL', 'function': {'arguments': '{\n  "id": "ai-304",\n  "title": "Gen AI applications",\n  "summary": "Exploring the emerging applications of gen AI.",\n  "author": "Sam K. Thompson",\n  "url": "https://medium.com/ai/gen-ai-applications"\n}', 'name': 'Publication'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 591, 'total_tokens': 652, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-C0g45DP4ZwqZavTVySQ5PeGZp6kbN', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6c01d955-978c-47e3-a590-b02a1208832d-0', tool_calls=[{'name': 'Publication', 'args': {'id': 'ai